> **Note:** This is a short notebook giving a quick taste of a concept that's also covered elsewhere in the course. It should be regarded as extra material. 

# Setup

In [ ]:
%matplotlib inline

import numpy as np, pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path
import seaborn as sns 
import sklearn
from sklearn import datasets

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

# Data

Vi bruker igjen datasettet om hus-priser kalt "California Housing Data Set". Det er som kjent bygget fra data samlet inn av U.S Census Service i 1990, og handler om huspriser i California. Det er gammelt, men mye brukt til illustrasjon av tema fra statistikk og maskinlæring. 

Beskrivelse sakset fra [artikkelen som introduserte datasettet](https://www.sciencedirect.com/science/article/pii/S016771529600140X):
> We collected information on the variables using all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. Naturally, the geographical area included varies inversely with the population density. We computed distances among the centroids of each block group as measured in latitude and longitude. We excluded all the block groups reporting zero entries for the independent and dependent variables. The final data contained 20,640 observations on 9 characteristics.

Denne gangen bruker vi versjonen som kommer innebygget i scikit-learn:

In [ ]:
california = datasets.fetch_california_housing(as_frame=True)

In [ ]:
print(california.DESCR)

In [ ]:
df = california.data

In [ ]:
df.info()

In [ ]:
df.describe()

Vi ser at det ikke virker å være noen missing values (strengt tatt bør dette undersøkes nøyere siden missing values kan være representert med tallverdier som f.eks. -1 eller lignende, slik beskrevet i notebooken om "imputation". Men i dette tilfellet vet vi fra beskrivelsen av datasettet at det ikke er noen missing values)

Som vanlig splitter vi opp features og target (som her er MedV -- median house value, lagret i `california.target`) i X og y, og lager oss et trenings- og test-sett:

In [ ]:
X,y = df.copy(), california.target

In [ ]:
X.head()

In [ ]:
len(y), y[:5]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Utforsk data

Som vanlig, etter å ha lastet ned data og tatt en kikk på hvordan data er representert så er neste naturlige steg å utforske og visualisere data. Vi skal ikke gå i detalj med dette nå da vi skal fokusere på feature engineering. Men, som nevnt, dette er en av de viktigste delene av praktisk maskinlæring, og en bruker gjerne en stor andel av tiden på slike utforskinger fordi det kan gi nyttig informasjon om problemstillingen man har satt opp er kompatibel med data en har tilgjengelig, og om hvordan en skal gå frem for å lage en god prediktiv modell (og også hva "god" skal bety). 

## Se etter korrelasjoner

La oss se etter lineære korrelasjoner mellom features, og også korrelasjoner mellom features og target. For å finne korrelasjoner mellom features og target lager vi oss en dataframe `Xy` som også inneholder target:

In [ ]:
Xy = X_train.copy()
Xy['MedV'] = y_train

In [ ]:
Xy.head()

In [ ]:
corr = Xy.corr()

Her er alle parvise korrelasjoner:

In [ ]:
corr

Her er korrelasjonene med target:

In [ ]:
corr['MedV'].sort_values(ascending=False)

Vi ser at MedInc ("median income in block") er ganske sterkt positivt korrelert med husverdiene (MEDV). Latitude er noe negativt korrelert med MEDV. Dette kan vi også plotte:

In [ ]:
Xy.plot.scatter('MedInc', 'MedV', figsize=(10,8))
plt.title('Median income versus MEDV')
plt.show()

Fra plottet ser vi at generelt vil høyere median-inntekt svare til høyere median-huspris i de ulike boligkvartalene. 

> Obs: vi observerer at det ser ut til å være satt et tak på 5 (altså $50.000) på median-husverdiene. Dette er noe en burde vurdert hvordan en best kan takle, hvis dette var et mer komplett maskinlæringsprosjekt-eksempel. Vi ser bort i fra dette her. 

Vi kan også la Pandas lage plots for alle par, og også histogram av feature-verdier.

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(Xy, figsize=(20, 20))
plt.show()

# Feature engineering

La oss forsøke å designe noen nye features ved å kombinere de vi allerede har. Dette kan gjøres ved å bruke en eller annen form for domenekunnskap (dvs. en idé om hva som vil være nyttige features for å predikere huspriser) og også ved å mer blindt forsøke feature-kombinasjoner. 

Som et eksempel på førstnevnte, kanskje andelen soverom i distriktet? Altså antall soverom delt på totalt antall rom. Det er naturlig å gjette på at dette er negativt korrelert med husprisene: jo flere rom som _ikke_ er soverom, jo dyrere hus. 

In [ ]:
Xy['AveBedrmsFraction'] = Xy['AveBedrms'] / Xy['AveRooms']

La oss teste hvordan dette er korrelert med husprisene:

In [ ]:
corr = Xy.corr()

In [ ]:
corr['MedV'].sort_values(ascending=False)

Vi ser at gjennomsnittelig andel soverom er vesentlig mer korrelert med huspriser enn både gjennomsnittelig antall rom og gjennomsnittelig antall soverom!

In [ ]:
Xy.plot.scatter('AveBedrmsFraction', 'MedV', figsize=(10,8))
plt.title('Andel soverom versus MedV')
plt.show()

> **Yor turn!** Kan du komme opp med andre, lignende features?

## Automatisk generering av features

En annen, mer "blind" fremgangsmåte er å generere for eksempel produkter av features, eller _polynomielle_ features for alle numeriske features. Altså nye features ved å multiplisere eksisterende features med hverandre og seg selv. 

In [ ]:
print(california.DESCR)

For noen av features kan dette tolkes fra beskrivelsene av de orginale features, f.eks. produktet AveOccup * Population = (antall hus / populasjon) * populasjon = antall hus, for andre er det mindre tolkbart.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X.head()

La oss lage alle polynom opp til andre grad. Altså $x_i, x_i^2, x_ix_j$ basert på alle features $x_i$.

> Hvor mange numeriske features får vi da?

> Vi hadde opprinnelig 8 numeriske features. Vi får da alle disse 8 + alle de 8 i andre potens + alle par $x_i x_j$ for alle ulike features. Det blir 8 + 8 + antall par av de åtte man kan velge, uten tilbakelegging = 8 + 8 + antall kombinasjoner:

In [ ]:
from math import comb

In [ ]:
8 + 8 + comb(8,2)

In [ ]:
polys = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
polyfeatures = polys.fit_transform(X_train)

In [ ]:
X_train_poly = pd.DataFrame(data=polyfeatures)

Vi har nå 44 features:

In [ ]:
X_train_poly.head()

In [ ]:
X_train_poly['MedV'] = y_train

In [ ]:
corr_poly = X_train_poly.corr()

Vi kan nå sjekke korrelasjoner med target: 

In [ ]:
corr_poly['MedV'].sort_values(ascending=False)